In [8]:
!pip install fog_x
!pip install tensorflow[and-cuda]
!pip install tensorflow_datasets


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 46.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 33.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=065b0237f470f3d985da7b209d3094b4ea457b0e3ed64c9cf8f7b0440c45cce6
  Stored in directory: /home/lifelong/.cache/pip/wheels/54/4e/28/3ed0e1c8a752867445bab994d2340

In [12]:
import fog_x 
import os
from logging import getLogger
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf

class BaseLoader():
    def __init__(self, data_path):
        super(BaseLoader, self).__init__()
        self.data_dir = data_path
        self.logger = getLogger(__name__)
        self.index = 0


    def __len__(self):
        raise NotImplementedError

    def __iter___(self):
        raise NotImplementedError

class BaseExporter():
    def __init__(self):
        super(BaseExporter, self).__init__()
        self.logger = getLogger(__name__)

    def export(self, loader: BaseLoader, output_path: str):
        raise NotImplementedError

In [13]:
class RTXLoader(BaseLoader):
    def __init__(self, data_path, split):
        super(RTXLoader, self).__init__(data_path)
        import tensorflow_datasets as tfds

        builder = tfds.builder_from_directory(data_path)

        self.ds = builder.as_dataset(split=split)
        # https://www.determined.ai/blog/tf-dataset-the-bad-parts

    def __len__(self):
        return len(self.ds)
    
    def __next__(self):
        if self.index < len(self.ds):
            self.index += 1
            nested_dataset = self.ds.__iter__()
            trajectory = list(nested_dataset)[0]["steps"]
            ret = []
            # Iterate through the outer dataset
            for step_data in trajectory:
                step = {}
                for dataset_key, element in step_data.items():
                    # print(np.array(element))
                    if dataset_key == "observation":
                        step["observation"] = {}
                        for obs_key, obs_element in element.items():
                            step["observation"][obs_key] = np.array(obs_element)
                    elif dataset_key == "action":
                        step["action"] = {}
                        for action_key, action_element in element.items():
                            step["action"][action_key] = np.array(action_element)
                    else:
                        step[dataset_key] = np.array(element)
                ret.append(step)
            return ret
        else:
            raise StopIteration
    

    
    def __iter__(self):
        return self


class RTXExporter(BaseExporter):
    def __init__(self):
        super(RTXExporter, self).__init__()

    def export(self, loader: BaseLoader, output_path: str):
        raise NotImplementedError

In [14]:
import h5py
class HDF5Loader(BaseLoader):
    def __init__(self, data_path):
        super(HDF5Loader, self).__init__(data_path)
        self.ds = []
        with h5py.File(data_path, "r") as f:
            print("Trajectories: %s" % f.keys())
            trajectories = []
            for trajectory_key in f.keys():
                traj_hdf5 = f[trajectory_key]
                print("Steps: %s" % traj_hdf5.keys())
                steps = []
                for step_key in traj_hdf5.keys():
                    step = {}
                    step_hdf5 = traj_hdf5[step_key]
                    for item_key,item_value in step_hdf5.items():
                        if(item_key == 'action'):
                            action_dict = {}
                            for action_key,action_value in item_value.items():
                                if(action_key == 'gripper_closedness_action'):
                                    action_dict['gripper_closedness_action'] = action_value[()]
                                elif(action_key == 'rotation_delta'):
                                    action_dict['rotation_delta'] = action_value[()]
                                elif(action_key == 'terminate_episode'):
                                    action_dict['terminate_episode'] = action_value[()]
                                elif(action_key == 'world_vector'):
                                    action_dict['world_vector'] = action_value[()]
                            step['action'] = action_dict
                        elif(item_key == 'is_first'):
                            step['is_first'] = item_value[()]
                        elif(item_key == 'is_last'):
                            step['is_last'] = item_value[()]
                        elif(item_key == 'is_terminal'):
                            step['is_terminal'] = item_value[()]
                        elif(item_key == 'observation'):
                            observation_dict = {}
                            for observation_key,observation_value in item_value.items():
                                if(observation_key == 'hand_image'):
                                    observation_dict['hand_image'] = observation_value[()]
                                elif(observation_key == 'image'):
                                    observation_dict['image'] = observation_value[()]
                                elif(observation_key == 'image_with_depth'):
                                    observation_dict['image_with_depth'] = observation_value[()]
                                elif(observation_key == 'natural_language_embedding'):
                                    observation_dict['natural_language_embedding'] = observation_value[()]
                                elif(observation_key == 'natural_language_instruction'):
                                    observation_dict['natural_language_instruction'] = observation_value[()]
                                elif(observation_key == 'robot_state'):
                                    observation_dict['robot_state'] = observation_value[()]
                            step['observation'] = observation_dict
                        elif(item_key == 'reward'):
                            step['reward'] = item_value[()]
                    steps.append(step)
                trajectories.append(steps)
            self.ds = trajectories
        
    def __len__(self):
        return len(self.ds)
    
    def __iter__(self):
        return self.ds.__iter__()
    
class HDF5Exporter(BaseExporter):
    def __init__(self):
        super(HDF5Exporter, self).__init__()

    def export(self, loader: BaseLoader, output_path: str):
        with h5py.File(output_path, 'w') as hdf:
            for i,trajectory in enumerate(loader):
                traj_key = "traj" + str(i).zfill(5)
                traj_hdf5 = hdf.create_group(traj_key)
                for j,data in enumerate(trajectory):
                    step_key = "step" + str(j).zfill(5)
                    step_hdf5 = traj_hdf5.create_group(step_key)
                    for key,value in data.items():
                        if(key == 'action'):
                            action_hdf5 = step_hdf5.create_group("action")
                            for action_key,action_value in value.items():
                                if(action_key == 'gripper_closedness_action'):
                                    action_hdf5.create_dataset('gripper_closedness_action',data=action_value)
                                elif(action_key == 'rotation_delta'):
                                    action_hdf5.create_dataset('rotation_delta',data=action_value)
                                elif(action_key == 'terminate_episode'):
                                    action_hdf5.create_dataset('terminate_episode',data=action_value)
                                elif(action_key == 'world_vector'):
                                    action_hdf5.create_dataset('world_vector',data=action_value)
                        elif(key == 'is_first'):
                            step_hdf5.create_dataset('is_first',data=np.array(value))
                        elif(key=='is_last'):
                            step_hdf5.create_dataset('is_last',data=value)
                        elif(key=='is_terminal'):
                            step_hdf5.create_dataset('is_terminal',data=value)
                        elif(key=='observation'):
                            observation_hdf5 = step_hdf5.create_group("observation")
                            for observation_key,observation_value in value.items():
                                if(observation_key == 'hand_image'):
                                    observation_hdf5.create_dataset('hand_image',data=observation_value)
                                elif(observation_key == 'image'):
                                    observation_hdf5.create_dataset('image',data=observation_value)
                                elif(observation_key == 'image_with_depth'):
                                    observation_hdf5.create_dataset('image_with_depth',data=observation_value)
                                elif(observation_key == 'natural_language_embedding'):
                                    observation_hdf5.create_dataset('natural_language_embedding',data=observation_value)
                                elif(observation_key == 'natural_language_instruction'):
                                    observation_hdf5.create_dataset('natural_language_instruction',data=observation_value)
                                elif(observation_key == 'robot_state'):
                                    observation_hdf5.create_dataset('robot_state',data=observation_value)
                        elif(key=='reward'):
                            step_hdf5.create_dataset('reward',data=value)

In [15]:
number_of_samples = 3
rtx_loader = RTXLoader(os.path.expanduser("/home/lifelong/fog_x/datasets/0.1.0"), split=f'train[:{number_of_samples}]')
hdf5_exporter = HDF5Exporter()
hdf5_exporter.export(rtx_loader,'please.hdf5')
hdf5_loader = HDF5Loader('/home/lifelong/fog_x/please.hdf5')
print(hdf5_loader.__len__())
for trajectory in hdf5_loader:
    for step in trajectory:
        print(step)

I 2024-06-23 14:37:21,456 dataset_info.py:617] Load dataset info from /home/lifelong/fog_x/datasets/0.1.0
I 2024-06-23 14:37:21,461 reader.py:261] Creating a tf.data.Dataset reading 1 files located in folders: /home/lifelong/fog_x/datasets/0.1.0.
I 2024-06-23 14:37:21,500 logging_logger.py:49] Constructing tf.data.Dataset berkeley_autolab_ur5 for split train[:3], from /home/lifelong/fog_x/datasets/0.1.0


Trajectories: <KeysViewHDF5 ['traj00000', 'traj00001', 'traj00002']>
Steps: <KeysViewHDF5 ['step00000', 'step00001', 'step00002', 'step00003', 'step00004', 'step00005', 'step00006', 'step00007', 'step00008', 'step00009', 'step00010', 'step00011', 'step00012', 'step00013', 'step00014', 'step00015', 'step00016', 'step00017', 'step00018', 'step00019', 'step00020', 'step00021', 'step00022', 'step00023', 'step00024', 'step00025', 'step00026', 'step00027', 'step00028', 'step00029', 'step00030', 'step00031', 'step00032', 'step00033', 'step00034', 'step00035', 'step00036', 'step00037', 'step00038', 'step00039', 'step00040', 'step00041', 'step00042', 'step00043', 'step00044', 'step00045', 'step00046', 'step00047', 'step00048', 'step00049', 'step00050', 'step00051', 'step00052', 'step00053', 'step00054', 'step00055', 'step00056', 'step00057', 'step00058', 'step00059', 'step00060', 'step00061', 'step00062', 'step00063', 'step00064', 'step00065', 'step00066', 'step00067', 'step00068', 'step00069',